## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [1]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


## 📦 Carregamento das Bibliotecas para a Fase 3: Modelagem Preditiva

Nesta etapa, carregamos todas as bibliotecas necessárias para realizar o treinamento, validação e interpretação de modelos de Machine Learning aplicados à previsão da tendência do Ibovespa.

---

### 🔧 Principais Componentes Importados:

#### 📊 Manipulação de Dados
- `pandas`, `numpy`: Estruturação e transformação de dados tabulares e numéricos.
- `duckdb`: Consulta e carregamento eficiente da base persistida na fase anterior.

#### ⚙️ Modelagem e Avaliação
- `lightgbm`: Framework de gradient boosting eficiente, usado para modelagem supervisionada.
- `sklearn.model_selection.train_test_split`: Divisão da base de forma temporal para simular previsão realista.
- `sklearn.metrics`: Avaliação com métricas como `accuracy`, `ROC AUC`, `confusion_matrix`.

#### 🧠 Interpretação do Modelo
- `shap`: Framework de interpretabilidade para entender a importância das features no modelo treinado.

#### 📈 Visualização
- `matplotlib`, `seaborn`: Criação de gráficos e análise visual dos resultados.

#### 🛠️ Configuração do Projeto
- `src.config`: Importa o caminho e demais parâmetros definidos nas fases anteriores.

---

✅ Todas as bibliotecas e dependências da **Fase 3 - Modelagem** foram carregadas com sucesso e estão prontas para uso.


In [2]:
import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from sklearn.model_selection import train_test_split # Usaremos para a divisão temporal
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# Importa nossas configurações de projeto
import src.config as config

# Helpers do Notebook
from IPython.display import display

# Configurações de estilo
sns.set_theme(style='whitegrid', palette='viridis')
plt.style.use("fivethirtyeight")
%matplotlib inline

print("✅ Bibliotecas para a Fase 3 carregadas com sucesso!")

c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Bibliotecas para a Fase 3 carregadas com sucesso!


## 🧱 Carregamento dos Dados para Modelagem

Nesta etapa, buscamos no banco de dados DuckDB a tabela `features_completas`, que contém todas as variáveis criadas e tratadas na **Fase 2** do projeto. Esses dados são a base para o treinamento dos modelos de Machine Learning.

---

### 📥 Etapas Realizadas:

1. **Conexão ao Banco DuckDB**
   - Utilizamos o caminho salvo no módulo `config`.

2. **Leitura da Tabela `features_completas`**
   - A tabela contém os dados finais após a engenharia de atributos, com a variável `alvo` (target) já definida.

3. **Conversão da Coluna `data`**
   - A coluna `data` é convertida para o tipo `datetime` e definida como índice do DataFrame.
   - Essa configuração é fundamental para **garantir uma divisão temporal correta** entre treino e teste, evitando vazamento de dados.

---

✅ Ao final desta célula, temos o DataFrame `df_model` carregado, indexado por data e pronto para os próximos passos de preparação e modelagem.


In [3]:
print(f"Carregando dados da tabela 'features_completas' de: {config.DB_PATH}")

try:
    con = duckdb.connect(database=str(config.DB_PATH), read_only=True)
    # MUITO IMPORTANTE: Selecionar da nova tabela com todas as features
    df_model = con.execute("SELECT * FROM features_completas").fetchdf()
    con.close()

    # Configura a coluna 'data' como o índice para facilitar a divisão temporal
    df_model['data'] = pd.to_datetime(df_model['data'])
    df_model.set_index('data', inplace=True)

    print("\n✅ Dados para modelagem carregados com sucesso!")
    print("Estrutura do DataFrame:")
    df_model.info()

except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar os dados: {e}")

Carregando dados da tabela 'features_completas' de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb

✅ Dados para modelagem carregados com sucesso!
Estrutura do DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2975 entries, 2014-02-05 to 2025-07-03
Data columns (total 83 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   close_petroleo_brent             2975 non-null   float64
 1   close_petrobras                  2975 non-null   float64
 2   close_dolar                      2975 non-null   float64
 3   close_ibovespa                   2975 non-null   float64
 4   close_sp500                      2975 non-null   float64
 5   high_petroleo_brent              2975 non-null   float64
 6   high_petrobras                   2975 non-null   float64
 7   high_dolar                       2975 non-null   float64
 8   high_ibovespa                    2975 non-null   

## 🎯 Criação da Variável Alvo Multiclasse

Para refinar o problema e permitir abordagens mais sofisticadas de modelagem, transformamos o alvo binário em uma **variável multiclasse com 3 categorias**, baseada na magnitude do retorno do dia seguinte do Ibovespa.

---

### 🧪 Lógica Utilizada

- **Classe 1 – Alta Significativa**: Retorno > +0.5%
- **Classe -1 – Baixa Significativa**: Retorno < -0.5%
- **Classe 0 – Neutro**: Variação entre -0.5% e +0.5%

Essa abordagem permite que o modelo diferencie entre movimentos significativos de mercado e ruídos de variação diária, tornando a previsão mais realista para aplicações práticas como alocação de risco e decisões de trading.

---

### 🔍 Resultado

A distribuição das classes no conjunto de dados foi verificada, garantindo equilíbrio e viabilidade para modelagem multiclasse. A última linha (que teria alvo indefinido) foi removida para manter a integridade do dataset.


In [4]:
# O DataFrame 'df_model' foi carregado e tem a data como índice.

print("--- Refinando o Problema: Criação do Alvo Multiclasse ---")

# 1. Definimos o nosso threshold de significância.
# Um movimento de 0.5% (0.005) é um bom ponto de partida.
# Podemos ajustar este valor mais tarde se necessário.
threshold = 0.005

# 2. Calculamos o retorno do dia seguinte para o Ibovespa
retorno_futuro = df_model['close_ibovespa'].pct_change().shift(-1)

# 3. Criamos a nova coluna 'alvo_multiclasse' com a lógica de 3 classes
# Usamos np.where aninhado, que funciona como um "SE/SENÃOSE/SENÃO"
df_model['alvo_multiclasse'] = np.where(
    retorno_futuro > threshold,      # Condição 1: Se o retorno futuro for > 0.5%
    1,                               # Então, a classe é 1 (Alta Significativa)
    np.where(
        retorno_futuro < -threshold, # Condição 2: Se o retorno futuro for < -0.5%
        -1,                          # Então, a classe é -1 (Baixa Significativa)
        0                            # Caso contrário, a classe é 0 (Neutra)
    )
)

# 4. Removemos o último dia, que terá um NaN no alvo
df_model.dropna(subset=['alvo_multiclasse'], inplace=True)

# --- Verificação ---
print("\nDistribuição do nosso novo alvo multiclasse (em %):")
# Usamos value_counts para ver quantas amostras temos de cada classe
print(df_model['alvo_multiclasse'].value_counts(normalize=True).sort_index().map('{:.2%}'.format))

print("\nExibindo as últimas linhas com o novo alvo para validação manual:")
display(df_model[['close_ibovespa', 'alvo_multiclasse']].tail(10))

--- Refinando o Problema: Criação do Alvo Multiclasse ---

Distribuição do nosso novo alvo multiclasse (em %):
alvo_multiclasse
-1    29.58%
 0    36.64%
 1    33.78%
Name: proportion, dtype: object

Exibindo as últimas linhas com o novo alvo para validação manual:


,close_ibovespa,alvo_multiclasse
data,,
2025-06-20,137116.0,0
2025-06-23,136551.0,0
2025-06-24,137165.0,-1
2025-06-25,135767.0,1
2025-06-26,137114.0,0
2025-06-27,136866.0,1
2025-06-30,138855.0,0
2025-07-01,139549.0,0
2025-07-02,139051.0,1
